In [ ]:
import pandas as pd #
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from scipy.stats import zscore

csv_path = "./data/Landslide_dataSet2.csv"
#device = "102"
#save_path = f"./data/dev{device}_prepared.csv"
save_path = f"data/dataSet2_prepared.csv"

df = pd.read_csv(csv_path)

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

#df = df[~df['timestamp'].str.contains(r'\(ICT\)', na=False)]
#df['timestamp'] = df['timestamp'].str.replace(r'\(ICT\)', '', regex=True).str.strip()

def clean_and_format_timestamp(val):
    if pd.isnull(val):
        return None
    val = str(val).replace('(ICT)', '').strip()
    try:
        dt = pd.to_datetime(val)
        return dt.isoformat()
    except Exception:
        return None  

df['timestamp'] = df['timestamp'].apply(clean_and_format_timestamp)

df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])

df = df.drop(columns=['Unnamed: 15', 'confidence','rf_risk'], errors='ignore')
df = df[(df['lat'] > 1.0) & (df['lat'] < 8000.0)]
df = df[df['lng'] > 15.0]
df = df[df['soil'] > 0]
df = df[df['rain'] >= 0]
df = df[df['humi'] > 0]
df = df[(df['devID'] == 104) | (df['devID'] == 103)]
df = df[(df['geo'] >= 0) & (df['geo'] < 40)]
df = df[df['temp'] > 0]
df = df[(df['devID'] > 0) & (df['devID'] < 1000)]
df = df[df['iso_score'] > -85000]
df = df[df['lstm_score'] >= 0]
#df = df[df['timestamp'] > '2025-09-11']

#df = df[df['devID'] == int(device)]
df

C:\Users\ahmad\AppData\Local\Temp\ipykernel_24568\3385453717.py:13: DtypeWarning: Columns (2,5,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


,timestamp,devID,soil,rain,temp,humi,geo,lat,lng,iso_score,iso_risk,lstm_score,lstm_risk
149609,2025-09-07 13:19:10.409,103,13.700000,0.0,27.299999,83.900002,0.000000,8.640431,99.911751,0.000000,0,0.000000,NaN
149610,2025-09-07 13:19:25.443,103,13.700000,0.0,27.299999,83.900002,0.000000,8.640431,99.911751,0.000000,0,0.000000,NaN
149611,2025-09-07 13:19:40.535,103,13.700000,0.0,27.200001,84.300003,0.000000,8.640431,99.911751,0.000000,0,0.000000,NaN
149612,2025-09-07 13:19:55.608,103,13.700000,0.0,27.200001,84.300003,0.000000,8.640431,99.911751,0.000000,0,0.000000,NaN
149613,2025-09-07 13:20:10.638,103,13.700000,0.0,27.200001,84.300003,0.000000,8.640431,99.911751,0.000000,0,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
260984,2025-10-21 02:45:07.164,104,19.700001,0.0,32.900002,63.599998,16.403057,8.640429,99.911758,0.013693,0,0.150991,0
260989,2025-10-21 02:45:59.409,103,20.100000,0.0,32.299999,62.500000,11.278135,8.640360,99.911819,0.030232,0,0.027337,0
260993,2025-10-21 02:47:00.750,104,19.700001,0.0,32.750000,62.849998,10.985930,8.640429,99.911758,0.032027,0,0.174430,0
260994,2025-10-21 02:47:11.154,103,20.100000,0.0,32.200001,64.099998,7.785539,8.640360,99.911819,0.037758,0,0.018421,0


In [52]:
df.describe()

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng,iso_score,iso_risk,lstm_score,lstm_risk
count,98361,98361.000000,98361.000000,98361.000000,98361.000000,98361.000000,98361.000000,98361.000000,98361.000000,98361.000000,98361.000000,98361.000000,89453.000000
mean,2025-09-26 21:08:01.995991552,103.485884,19.225505,0.066554,28.073332,80.572822,6.660490,8.639861,100.539439,7.867177,0.050721,0.266201,0.007412
min,2025-09-07 13:19:10.409000,103.000000,1.598425,0.000000,6.550000,24.228929,0.000000,1.135011,32.000000,-0.185142,0.000000,0.000000,0.000000
25%,2025-09-14 17:08:49.294000128,103.000000,17.700001,0.000000,25.299999,70.199997,0.431939,8.640358,99.911743,-0.005704,0.000000,0.015828,0.000000
50%,2025-09-22 09:37:32.644999936,103.000000,18.600000,0.000000,26.849999,84.300003,2.644981,8.640408,99.911774,0.014227,0.000000,0.041517,0.000000
75%,2025-10-08 17:51:14.537999872,104.000000,20.200001,0.000000,31.100000,92.000000,12.091402,8.640432,99.911812,0.054729,0.000000,0.106805,0.000000
max,2025-10-21 02:47:52.495000,104.000000,27.100000,36.321999,37.050001,98.800003,37.760407,8.640481,9954.666992,1760.843822,2.000000,187.449081,2.000000
std,NaN,0.499803,2.575367,0.801378,3.371179,13.320037,7.945585,0.063314,64.442908,59.097012,0.238308,3.860938,0.110290


In [53]:
df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])  # ต้องมี timestamp และ geo

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng', 'iso_risk', 'iso_score', 'lstm_risk']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng,iso_score,iso_risk,lstm_score,lstm_risk
0,2025-09-07 13:19:10.409,103,13.700000,0.0,27.299999,83.900002,0.000000,8.640431,99.911751,0.000000,0,0.000000,NaN
1,2025-09-07 13:19:25.443,103,13.700000,0.0,27.299999,83.900002,0.000000,8.640431,99.911751,0.000000,0,0.000000,NaN
2,2025-09-07 13:19:40.535,103,13.700000,0.0,27.200001,84.300003,0.000000,8.640431,99.911751,0.000000,0,0.000000,NaN
3,2025-09-07 13:19:55.608,103,13.700000,0.0,27.200001,84.300003,0.000000,8.640431,99.911751,0.000000,0,0.000000,NaN
4,2025-09-07 13:20:10.638,103,13.700000,0.0,27.200001,84.300003,0.000000,8.640431,99.911751,0.000000,0,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98356,2025-10-21 02:45:07.164,104,19.700001,0.0,32.900002,63.599998,16.403057,8.640429,99.911758,0.013693,0,0.150991,0.0
98357,2025-10-21 02:45:59.409,103,20.100000,0.0,32.299999,62.500000,11.278135,8.640360,99.911819,0.030232,0,0.027337,0.0
98358,2025-10-21 02:47:00.750,104,19.700001,0.0,32.750000,62.849998,10.985930,8.640429,99.911758,0.032027,0,0.174430,0.0
98359,2025-10-21 02:47:11.154,103,20.100000,0.0,32.200001,64.099998,7.785539,8.640360,99.911819,0.037758,0,0.018421,0.0


In [54]:
# เรียงลำดับตาม devID และ timestamp
df = df.sort_values('timestamp').reset_index(drop=True)
df = df.set_index('timestamp')
#df['hour'] = df.index.hourผ
#df['day_of_week'] = df.index.dayofweek
df = df.reset_index() 

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98361 entries, 0 to 98360
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   timestamp   98361 non-null  datetime64[ns]
 1   devID       98361 non-null  int64         
 2   soil        98361 non-null  float64       
 3   rain        98361 non-null  float64       
 4   temp        98361 non-null  float64       
 5   humi        98361 non-null  float64       
 6   geo         98361 non-null  float64       
 7   lat         98361 non-null  float64       
 8   lng         98361 non-null  float64       
 9   iso_score   98361 non-null  float64       
 10  iso_risk    98361 non-null  int64         
 11  lstm_score  98361 non-null  float64       
 12  lstm_risk   89453 non-null  float64       
dtypes: datetime64[ns](1), float64(10), int64(2)
memory usage: 9.8 MB


In [56]:
nan_counts = df.isna().sum()

nan_columns = nan_counts[nan_counts > 0]

print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
lstm_risk    8908
dtype: int64


In [57]:
#df = df.dropna(subset=['humi']).reset_index(drop=True)

In [58]:
nan_counts = df.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
lstm_risk    8908
dtype: int64


In [59]:
#df = df.dropna()
#
#nan_counts = df.isna().sum()
#nan_columns = nan_counts[nan_counts > 0]
#print("df102")
#print("📌 คอลัมน์ที่มีค่า NaN:")
#print(nan_columns)

In [60]:
#df[(df["iso_risk"]==2) & (df["lstm_risk"]==2)]

In [61]:
df.describe()

,timestamp,devID,soil,rain,temp,humi,geo,lat,lng,iso_score,iso_risk,lstm_score,lstm_risk
count,98361,98361.000000,98361.000000,98361.000000,98361.000000,98361.000000,98361.000000,98361.000000,98361.000000,98361.000000,98361.000000,98361.000000,89453.000000
mean,2025-09-26 21:08:01.995991552,103.485884,19.225505,0.066554,28.073332,80.572822,6.660490,8.639861,100.539439,7.867177,0.050721,0.266201,0.007412
min,2025-09-07 13:19:10.409000,103.000000,1.598425,0.000000,6.550000,24.228929,0.000000,1.135011,32.000000,-0.185142,0.000000,0.000000,0.000000
25%,2025-09-14 17:08:49.294000128,103.000000,17.700001,0.000000,25.299999,70.199997,0.431939,8.640358,99.911743,-0.005704,0.000000,0.015828,0.000000
50%,2025-09-22 09:37:32.644999936,103.000000,18.600000,0.000000,26.849999,84.300003,2.644981,8.640408,99.911774,0.014227,0.000000,0.041517,0.000000
75%,2025-10-08 17:51:14.537999872,104.000000,20.200001,0.000000,31.100000,92.000000,12.091402,8.640432,99.911812,0.054729,0.000000,0.106805,0.000000
max,2025-10-21 02:47:52.495000,104.000000,27.100000,36.321999,37.050001,98.800003,37.760407,8.640481,9954.666992,1760.843822,2.000000,187.449081,2.000000
std,NaN,0.499803,2.575367,0.801378,3.371179,13.320037,7.945585,0.063314,64.442908,59.097012,0.238308,3.860938,0.110290


In [62]:
df['devID'].value_counts()

devID
103    50569
104    47792
Name: count, dtype: int64

In [63]:

df.to_csv(f"{save_path}", index=False)

print("✅ บันทึกเรียบร้อยแล้ว")

✅ บันทึกเรียบร้อยแล้ว
